# Exploratory Analysis

Exploratory data analysis, or EDA, builds intuition from your data.  This lesson introduces the objectives and strategies for EDA including summary statistics, basic plotting, and correlations.

## ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) In this lesson you:<br>
* Identify the main objectives of exploratory analysis
* Calculate statistical moments to determine the center and spread of data
* Create plots of data including histograms and scatterplots
* Calculate correlations between variables
* Explore more advanced plots to visualize the relation between variables

<iframe  
src="//fast.wistia.net/embed/iframe/baptj0tmzh?videoFoam=true"
style="border:1px solid #1cb1c2;"
allowtransparency="true" scrolling="no" class="wistia_embed"
name="wistia_embed" allowfullscreen mozallowfullscreen webkitallowfullscreen
oallowfullscreen msallowfullscreen width="640" height="360" ></iframe>
<div>
<a target="_blank" href="https://fast.wistia.net/embed/iframe/baptj0tmzh?seo=false">
  <img alt="Opens in new tab" src="https://files.training.databricks.com/static/images/external-link-icon-16x16.png"/>&nbsp;Watch full-screen.</a>
</div>

-sandbox
### Exploratory Data Analysis

The goal of exploratory data analysis (EDA) is to build the intuition into a dataset that will inform how you model the data.  Models are only as strong as data that is fed into them and the more insight a data scientist has into their data, the stronger the features they'll create are and the more informed their design choices will be.

<div><img src="https://files.training.databricks.com/images/eLearning/ML-Part-1/eda.png" style="height: 500px; margin: 20px"/></div>

Exploratory analysis focuses on the following:<br><br>

1. Gain basic intuition into the data
  - What does each feature represent?
  - Are your features categorical or continuous?
  - What data needs to be encoded? (e.g. mapping a variable for gender to a number)
  - What data types are you working with? (e.g. integers, strings)
2. How is the data distributed?
  - What is the mean, median, and/or mode of each variable?
  - What is the variance, or spread, of each variable?
  - Are there outliers?  Missing values?
3. How can we visualize the data?
  - Histograms, scatterplots, and boxplots
  - Correlation plots
4. What hypotheses will I test with my models?
  - What features are correlated?
  - What are our ideal features and how can we build them if they're not already available?

Run the following cell to set up our environment.

In [5]:
%run "./Includes/Classroom-Setup"

-sandbox
### Count, Mean, and Standard Deviation

One way to start the EDA process is to calculate **n**, or the number of observations in the dataset, as well as the center and spread of the data.  We can break this down in the following way:<br><br>

1. Count gives us the number of observed values, indicating the size of the dataset and whether there are missing values.
2. Mean gives us the center of the data.  This could also be accomplished with the median or mode, depending on the data.
3. Standard deviation quantifies how spread out the data is from the mean.  A small standard deviation indicates that the data is closely centered on the mean.

<div><img src="https://files.training.databricks.com/images/eLearning/ML-Part-1/standard-deviation.png" style="height: 250px; margin: 20px"/></div>

This chart indicates how to interpret standard deviation.  68% of the data is within one standard deviation (represented by `σ`) from the mean.

<img alt="Side Note" title="Side Note" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.05em; transform:rotate(15deg)" src="https://files.training.databricks.com/static/images/icon-note.webp"/> See the <a href="https://en.wikipedia.org/wiki/Standard_deviation" target="_blank">Wikipedia article on Standard Deviation for more details on standard deviation.</a>

Import the Boston dataset and call the `.describe()` method on it.

In [8]:
bostonDF = (spark.read
  .option("HEADER", True)
  .option("inferSchema", True)
  .csv("/mnt/training/bostonhousing/bostonhousing/bostonhousing.csv")
  .drop("_c0")
)

display(bostonDF.describe())

summary,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat,medv
count,506,506,506,506,506,506,506,506,506,506,506,506,506,506
mean,3.6135235573122535,11.363636363636363,11.136778656126504,0.0691699604743083,0.5546950592885372,6.284634387351787,68.57490118577078,3.795042687747034,9.549407114624506,408.2371541501976,18.455533596837967,356.67403162055257,12.653063241106723,22.532806324110698
stddev,8.601545105332491,23.32245299451514,6.860352940897589,0.2539940413404101,0.11587767566755584,0.7026171434153232,28.148861406903595,2.10571012662761,8.707259384239366,168.53711605495903,2.1649455237144455,91.29486438415782,7.141061511348571,9.197104087379815
min,0.00632,0.0,0.46,0,0.385,3.561,2.9,1.1296,1,187,12.6,0.32,1.73,5.0
max,88.9762,100.0,27.74,1,0.871,8.78,100.0,12.1265,24,711,22.0,396.9,37.97,50.0


This shows that there are 506 observations in our dataset with no missing values since the counts of our records are all equal.  Take a look at just the output variable `medv`.

In [10]:
display(bostonDF.select("medv").describe())

summary,medv
count,506
mean,22.532806324110698
stddev,9.197104087379815
min,5.0
max,50.0


-sandbox
As seen with the mean and standard deviation, the average house price is $22,532 with 68% of the observations within $9,197 of that average.

<img alt="Side Note" title="Side Note" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.05em; transform:rotate(15deg)" src="https://files.training.databricks.com/static/images/icon-note.webp"/> `medv` is in 1000's of dollars

-sandbox
### Plotting, Distributions, and Outliers

Anscombe's Quartet is a set of four datasets that have identical means, standard deviations, and correlations.  Despite the fact that the descriptive statistics are the same, the underlying data is very different.  This is a cautionary tale for why data should always be plotted in the EDA phase.

<div><img src="https://files.training.databricks.com/images/eLearning/ML-Part-1/anscombes_quartet.svg" style="height: 400px; margin: 20px"/></div>

Plotting of a single variable is normally first done using a histogram.  This plots the general distribution of the data with the value on the x axis and the frequency the value appears on the y axis.  Plotting of two variables is normally first done with a scatterplot.

<img alt="Side Note" title="Side Note" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.05em; transform:rotate(15deg)" src="https://files.training.databricks.com/static/images/icon-note.webp"/> For plotting a single variable in a more more rigorous way, use <a href="https://en.wikipedia.org/wiki/Kernel_density_estimation" target="_blank">kernel density estimation</a> plots instead of histograms.

-sandbox
Plot a histogram of the median value in the housing dataset.  Display this column, click on the plotting option, and visualize it as a histogram.

<div><img src="https://files.training.databricks.com/images/eLearning/ML-Part-1/plotting.png" style="height: 300px; margin: 20px"/></div>

In [14]:
display(bostonDF.select("medv"))

medv
24.0
21.6
34.7
33.4
36.2
28.7
22.9
27.1
16.5
18.9


This plot shows the general distribution of median housing prices divided into 20 bins.  Now take a look at how the number of rooms compares to the median value.  Do this by selecting the scatterplot option from the plotting dropped menu.

In [16]:
display(bostonDF.select("rm", "medv"))

rm,medv
6.575,24.0
6.421,21.6
7.185,34.7
6.998,33.4
7.147,36.2
6.43,28.7
6.012,22.9
6.172,27.1
5.631,16.5
6.004,18.9


By clicking the "LOESS" box under Plot Options, a local regression line is added showing the trend.

-sandbox
Databricks notebooks can display plots generated in pure Python:<br><br>

1. Import `matplotlib`, `pandas`, and `seaborn`
2. Create a `fig` object
3. Use the `.toPandas()` DataFrame method to turn the Spark DataFrame into a Pandas DataFrame.  This way we can use Python's plotting libraries
4. Use the `scatter_matrix` pandas function to plot a matrix of scatterplots

<img alt="Caution" title="Caution" style="vertical-align: text-bottom; position: relative; height:1.3em; top:0.0em" src="https://files.training.databricks.com/static/images/icon-warning.svg"/> Do not use `.toPandas()` on large datasets since a Pandas DataFrame must fit into the driver node of a cluster.

Plot a scatter matrix using the `pandas` Python library.

In [20]:
%python
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns # importing for a better color scheme

try:
  bostonDF
except NameError: # Looks for local table if bostonDF not defined
  bostonDF = spark.table("boston")

fig, ax = plt.subplots()
pandasDF = bostonDF.select("rm", "crim", "lstat", "medv").toPandas()

pd.plotting.scatter_matrix(pandasDF)

display(fig.figure)

The above plot shows histograms on the diagonal and scatterplots in the other quadrants.  The plot in the upper-left hand quarter is a histogram of the `rm` variable and the plot to its right is a scatterplot with `rm` as the y axis and `crim` as the x axis.  This shows us correlations across our variables.

-sandbox
### Correlations

Correlation is the first way of understanding the relationship between two variables.  The correlation of two variables is the degree to which one variable increases linearly with the other:<br><br>

* A positive correlation of 1 means that for each unit increase in one variable, the same increase is seen in the other
* A negative correlation of -1 means that for each unit increase in one variable, the same decrease is seen in the other
* A correlation of 0 means that there is no association between the variables

More formally, correlation is computed as the following:<br><br>

<div><img src="https://files.training.databricks.com/images/eLearning/ML-Part-1/correlation.svg" style="height: 60px; margin: 20px"/></div>

The correlation of variables X and Y is their covariance over the standard deviation of X multiplied by the standard deviation of Y.  The covariance is calculated by summing up the product of each value minus their respective mean (`μ`).

To deepen our understanding of correlation, add columns to the `bostonDF` DataFrame that accomplishes the following:<br><br>

1. `medvX3`: `medv` multiplied by 3
3. `medvNeg`: `medv` multiplied by -3
4. `random1`: a random number
5. `random2`: a second random number
6. `medvWithNoise`: `medv` multiplied by some random noise
7. `medvWithNegativeNoise`: negative `medv` multiplied by some random noise

In [24]:
from pyspark.sql.functions import col, rand

bostonWithDummyDataDF = (bostonDF
  .select("medv")
  .withColumn("medvX3", col("medv")*3)
  .withColumn("medvNeg", col("medv")*-3)
  .withColumn("random1", rand(seed=41))
  .withColumn("random2", rand(seed=44))
  .withColumn("medvWithNoise", col("medv")*col("random1"))
  .withColumn("medvWithNegativeNoise", col("medv")*col("random1")*-1)
)

display(bostonWithDummyDataDF)

medv,medvX3,medvNeg,random1,random2,medvWithNoise,medvWithNegativeNoise
24.0,72.0,-72.0,0.9502423797032892,0.7985334687187781,22.80581711287894,-22.80581711287894
21.6,64.80000000000001,-64.80000000000001,0.8724902006601648,0.8679617292463396,18.84578833425956,-18.84578833425956
34.7,104.10000000000001,-104.10000000000001,0.5469560487041895,0.5068407664758031,18.979374890035377,-18.979374890035377
33.4,100.19999999999999,-100.19999999999999,0.45521698538104904,0.7946993123364816,15.204247311727038,-15.204247311727038
36.2,108.60000000000001,-108.60000000000001,0.3726668369477889,0.7463770360606422,13.490539497509959,-13.490539497509959
28.7,86.1,-86.1,0.8102793186273827,0.27140710651938027,23.255016444605882,-23.255016444605882
22.9,68.69999999999999,-68.69999999999999,0.08157144775154124,0.2741303337234684,1.8679861535102942,-1.8679861535102942
27.1,81.30000000000001,-81.30000000000001,0.7895113419776418,0.8373499476137246,21.395757367594094,-21.395757367594094
16.5,49.5,-49.5,0.840914329463114,0.24589255141259847,13.87508643614138,-13.87508643614138
18.9,56.699999999999996,-56.699999999999996,0.5129710094443797,0.9652351351138434,9.695152078498776,-9.695152078498776


Calculate correlations between all columns and `id`

In [26]:
for col in bostonWithDummyDataDF.columns:
  correlation = bostonWithDummyDataDF.stat.corr("medv", col)
  
  print("The correlation between columns 'id' and '{}': \t{}".format(col, correlation))

The correlation between columns 'id' and 'medv': 1.0
The correlation between columns 'id' and 'medvX3': 1.0000000000000004
The correlation between columns 'id' and 'medvNeg': -1.0000000000000004
The correlation between columns 'id' and 'random1': 0.006432271130492889
The correlation between columns 'id' and 'random2': -0.05810370320527292
The correlation between columns 'id' and 'medvWithNoise': 0.5446845136944051
The correlation between columns 'id' and 'medvWithNegativeNoise': -0.5446845136944051

The main takeaways:<br><br>

* The correlation between `id` and itself is 1, as is the correlation between `id` and `id` multiplied by 3.  
* The correlation between `id` and `id` multiplied by -3 is -1.  
* The correlations between `id` and random noise varied based on what was in the noise, but the results were closer to 0.  
* There were stronger positive and negative correlations in random noise multiplied by `id` than in random noise not correlated to `id`

### Other Visualization Tools

There are a number of other helpful visualizations depending on the needs of your data.  These include:<br><br>

* <a href="https://en.wikipedia.org/wiki/Heat_map" target="_blank">Heat maps:</a> similar to a scattermatrix, heatmaps can be especially helpful at visualizing correlations between variables
* <a href="https://en.wikipedia.org/wiki/Box_plot" target="_blank">Box plots:</a> visualizes quantiles and outliers
* <a href="https://en.wikipedia.org/wiki/Q%E2%80%93Q_plot" target="_blank">Q-Q Plots:</a> visualizes two probability distributions
* <a href="https://en.wikipedia.org/wiki/Geographic_information_system" target="_blank">Maps and GIS:</a> visualizes geographically-bound data
* <a href="https://en.wikipedia.org/wiki/T-distributed_stochastic_neighbor_embedding" target="_blank">t-SNE:</a> plots high dimensional data (i.e. data that has many variables) by projecting it down into two-diminsional plot
* <a href="https://en.wikipedia.org/wiki/Time_series" target="_blank">Time series:</a> plots time-bound variables including run charts, lag plots, and wavelet spectrograms

Assemble all of the `bostonDF` features into a single column `features`.  This allows us to use Spark's built-in correlation functionality.

In [30]:
%python
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=bostonDF.columns, outputCol="features")

bostonFeaturizedDF = assembler.transform(bostonDF)

Calculate the correlations across the entire dataset.

In [32]:
%python
from pyspark.ml.stat import Correlation

pearsonCorr = Correlation.corr(bostonFeaturizedDF, 'features').collect()[0][0]
pandasDF = pd.DataFrame(pearsonCorr.toArray())

pandasDF.index, pandasDF.columns = bostonDF.columns, bostonDF.columns # Labels our index and columns so we can interpret the results

/databricks/spark/python/pyspark/sql/context.py:119: DeprecationWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
 DeprecationWarning)

Plot a heatmap of the correlations.  The redder the value, the stronger the positive correlation and the bluer the value the stronger the negative correlation.  Do this in pure Python rather than Spark because Python has better plotting functionality.

In [34]:
%python
import matplotlib.pyplot as plt
import seaborn as sns

fig, ax = plt.subplots()
sns.heatmap(pandasDF)
display(fig.figure)